# Учебный Проект → «Викишоп» 📚

>Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Импортируем библиотеки 🎒

In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from numpy.random import RandomState

from matplotlib import rcParams

from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from catboost import CatBoostRegressor

Настроим вид графиков по+красоте ✨

In [2]:
%config InlineBackend.figure_formats = ['svg']

In [3]:
# style MATPLOTLIBRC
custom_params = {
                'figure.figsize': (10, 6),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,
    
                "lines.color": '#EEEEEE',

                'text.color': '#EEEEEE',
    
                'font.family': 'sans-serif',
            }

In [4]:
# set max columns to none
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

Константы.

In [5]:
RANDOM_SEED = 108108108
RANDOM_NP = RandomState(128)

Функции.

<hr>

## Загрузим данные

In [29]:
try:
    comments_df = pd.read_csv(
        './datasets/toxic_comments.csv',
        usecols=['text', 'toxic']
    )
    
except FileNotFoundError:
    comments_df = pd.read_csv(
        'https://code.s3.yandex.net/datasets/toxic_comments.csv',
        usecols=['text', 'toxic']
    )
    print('FYI datasets loaded via url')

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

In [9]:
display(
    comments_df.head(2),
    comments_df.sample(2),
    comments_df.tail(2)
    
)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0


,text,toxic
36780,Thanks for the reply. I perceive that I am th...,0
74842,I fixed it and added a legal video source stra...,0


,text,toxic
159290,And it looks like it was actually you who put ...,0
159291,"""\nAnd ... I really don't think you understand...",0


Загрузите и подготовьте данные.
Обучите разные модели.
Сделайте выводы.

import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
df_tweets = pd.read_csv('/datasets/tweets.csv')

In [ ]:
tokenizer = transformers.BertTokenizer(
    vocab_file='/datasets/ds_bert/vocab.txt')

tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
config = transformers.BertConfig.from_json_file(
    '/datasets/ds_bert/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '/datasets/ds_bert/rubert_model.bin', config=config)

In [ ]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))

data = pd.read_csv("/datasets/tweets_lemm.csv")
corpus = data['lemm_text'].values.astype('U')

# создайте мешок слов без учёта стоп-слов
count_vect = CountVectorizer()
bow = count_vect.fit_transform(corpus)

print("Размер мешка без учёта стоп-слов:", bow.shape)

# создайте новый мешок слов с учётом стоп-слов
count_vect = CountVectorizer(stop_words=stop_words)
bow = count_vect.fit_transform(corpus)

print("Размер мешка с учётом стоп-слов:", bow.shape)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("/datasets/tweets_lemm.csv")
corpus = list(data['lemm_text'])

# создайте n-грамму n_gramm, для которой n=2
count_vect = CountVectorizer(ngram_range=(2, 2))
n_gramm = count_vect.fit_transform(corpus)
print("Размер:", n_gramm.shape)

$$ \large
TF = \dfrac{t}{n}
$$

>где t (от англ. term) — количество употребления слова, а n — общее число слов в тексте.  
IDF нужна в формуле, чтобы уменьшить вес слов, наиболее распространённых в любом другом тексте заданного корпуса. IDF зависит от общего числа текстов в корпусе (D) и количества текстов, в которых это слово встречается (d). 

$$ \large
IDF = \log_{10}\dfrac{D}{d}
$$

$$ \large
TFIDF = TF * IDF
$$

> Большая величина TF-IDF говорит об уникальности слова в тексте по отношению к корпусу.  
Чем чаще оно встречается в конкретном тексте и реже в остальных, тем выше значение TF-IDF.

In [10]:
text = [
    'Карл у Клара украсть коралл',
    'Клара у Карл украсть кларнет',
    'Клара строго карать Карл за кража коралл'
]

In [26]:
tf = 1/(len(text[2].split()) - 1)
print(tf)

0.16666666666666666


In [20]:
poem = []
poem.extend(text[0].split())
poem.extend(text[1].split())
poem.extend(text[2].split())
display(set(poem))
display(len(set(poem)))

{'Карл',
 'Клара',
 'за',
 'карать',
 'кларнет',
 'коралл',
 'кража',
 'строго',
 'у',
 'украсть'}

10

In [28]:
import math
 
tf = 1 / len(text[2].split())

idf = math.log(3/2,10)
print (idf)
tfidf = tf * idf
print (tfidf)

0.17609125905568124
0.025155894150811604


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

data = pd.read_csv("/datasets/tweets_lemm.csv")
corpus = data['lemm_text'].values.astype('U')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

print("Размер матрицы:", tf_idf.shape)

In [30]:
tweets_lemm_train = pd.read_csv(
        './datasets/tweets_lemm_train.csv',
    )

tweets_lemm_test= pd.read_csv(
        './datasets/tweets_lemm_test.csv',
    )

>Все ли шаги по инструкции выполняете?  
Как готовите данные?  
Как анализируете данные?  
Какие модели и гиперпараметры рассматриваете?  
Не дублируете ли код?  
Какие выводы делаете?  
Следите ли за структурой проекта?  
Поддерживаете аккуратность кода?  